In [ ]:
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
with open(f'/content/drive/MyDrive/final.pkl', 'rb') as f:
  target_texts = pickle.load(f)

In [ ]:
len(target_texts)

300000

In [ ]:
import unicodedata

REMOVE_DIACRITIC_TABLE = str.maketrans(
    "ÁÀẢÃẠĂẮẰẲẴẶÂẤẦẨẪẬĐÈÉẺẼẸÊẾỀỂỄỆÍÌỈĨỊÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÚÙỦŨỤƯỨỪỬỮỰÝỲỶỸỴáàảãạăắằẳẵặâấầẩẫậđèéẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵ",
    "A" * 17 + "D" + "E" * 11 + "I" * 5 + "O" * 17 + "U" * 11 + "Y" * 5 + "a" * 17 + "d" + "e" * 11 + "i" * 5 + "o" * 17 + "u" * 11 + "y" * 5
)


def remove_diacritic(txt: str) -> str:
  if not unicodedata.is_normalized("NFC", txt):
    txt = unicodedata.normalize("NFC", txt)
  return txt.translate(REMOVE_DIACRITIC_TABLE)

print(remove_diacritic("Đà Nẵng"))

Da Nang


In [ ]:
input_texts = [remove_diacritic(sentence) for sentence in target_texts]

In [ ]:
input_texts[1:-1:50000]

['Giam can chi don thuan la giam di trong luong co the co the tu viec an kieng nhin an hoac tinh trang benh ly nao do',
 'Co nguoi con kem theo cac trieu chung toan than nhu met moi buon ngu phu mat tu tuong tam trang khong on dinh',
 'Cach mang mau voi nhieu ten goi khac nhau nhu',
 'Ong Tien co mot nguoi ban ten la Duoc co can nha lau sat ben canh',
 'Toi co cam tuong nhu phim duoc lam rat thoang co cam hung tu nhien nhu mot luong gio moi chua day oxy ma khan gia co the hit day long nguc',
 'Can cu Hien phap nuoc Cong hoa xa hoi chu nghia Viet Nam;']

In [ ]:
# Lower the text from input and output
input_texts = [txt.lower() for txt in input_texts]
target_texts = [txt.lower() for txt in target_texts]

In [ ]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# Tokenize the inputs
MAX_NUM_WORDS = 30000
tokenizer_input = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_input.fit_on_texts(input_texts)
input_sequences = tokenizer_input.texts_to_sequences(input_texts)

In [ ]:
# Get the word to index mapping for input
word2idx_input = tokenizer_input.word_index
print('Found %s unique tokens.' % len(word2idx_input))

Found 22219 unique tokens.


In [ ]:
# determine max length input sequence
max_len_input = max(len(s) for s in input_sequences)
print('Max input length:', max_len_input)

Max input length: 411


In [ ]:
# pad the sequences to NxT matrix
input_sequences = pad_sequences(input_sequences, maxlen=max_len_input, padding='post', value=0)
print('Shape of input tensor:', input_sequences.shape)

Shape of input tensor: (300000, 411)


In [ ]:
# Create a mapping from targets to target labels
import re
def remove_punctuation(text: str) -> str:
    return re.sub(r'[^\w\s]', '', text)

In [ ]:
target_texts = [remove_punctuation(txt) for txt in target_texts]

In [ ]:
word2idx = {}
for text in target_texts:
  data = text.split(' ')
  for word in data:
    remove_diacritic_word = remove_diacritic(word)
    if remove_diacritic_word not in word2idx:
      word2idx[remove_diacritic_word] = {word: 1}
    else:
      if word not in word2idx[remove_diacritic_word]:
        word2idx[remove_diacritic_word][word] = len(word2idx[remove_diacritic_word]) + 1

In [ ]:
# Tokenizet the output
MAX_NUM_WORDS = 60000
tokenizer_target = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer_target.fit_on_texts(target_texts)

In [ ]:
# Get the word to index mapping for target
word2idx_target = tokenizer_target.word_index
print('Found %s unique tokens.' % len(word2idx_target))

Found 25592 unique tokens.


In [ ]:
target_labels = []
for text in target_texts:
  label = []
  data = text.split(' ')
  for word in data:
    remove_diacritic_word = remove_diacritic(word)
    label.append(word2idx[remove_diacritic_word][word])
  target_labels.append(label)

In [ ]:
target_labels = pad_sequences(target_labels, maxlen=max_len_input, padding='post', value=0)
print('Shape of target tensor:', target_labels.shape)

Shape of target tensor: (300000, 411)


In [ ]:
num_labels = max(max(label) for label in target_labels) + 1

In [ ]:
num_labels

30

Create an index-to-word Dictionary

In [ ]:
idx2word = {}
for key in word2idx.keys():
  idx2word[key] = {}
  for word in word2idx[key].keys():
    idx2word[key][word2idx[key][word]] = word

Save the input and output tokenizer

In [ ]:
import pickle
with open(f'tokenizer_input_{style}.pkl', 'wb') as handle:
  pickle.dump(tokenizer_input, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(f'tokenizer_target_{style}.pkl', 'wb') as handle:
  pickle.dump(tokenizer_target, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(f'idx2word_{style}.pkl', 'wb') as handle:
  pickle.dump(idx2word, handle, protocol=pickle.HIGHEST_PROTOCOL)

Deploy the model

In [ ]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional

In [ ]:
num_words = min(MAX_NUM_WORDS, len(word2idx_input) + 1)

In [ ]:
num_words

22220

In [ ]:
EMBEDDING_DIM = 200
# create embedding layer
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    input_length=max_len_input,
    trainable=True
)

In [ ]:
input = Input(shape=(max_len_input,))
x = embedding_layer(input)
model = SpatialDropout1D(0.1)(x)
model = Bidirectional(LSTM(units=50, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(num_labels, activation="softmax"))(model)
model = Model(input, out)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 411)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 411, 200)          4444000   
                                                                 
 spatial_dropout1d (Spatial  (None, 411, 200)          0         
 Dropout1D)                                                      
                                                                 
 bidirectional (Bidirection  (None, 411, 100)          100400    
 al)                                                             
                                                                 
 time_distributed (TimeDist  (None, 411, 30)           3030      
 ributed)                                                        
                                                             

In [ ]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

Train the model

In [ ]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    f'BiLSTM_model.h5', verbose=1, save_weights_only=True)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=3, verbose=1)

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10
VALIDATION_SPLIT = 0.2

In [ ]:
print('Training model...')
r = model.fit(
    input_sequences,
    target_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=VALIDATION_SPLIT,
    callbacks = [cp_callback, es_callback]
)

Training model...
Epoch 1/10
313/313 [==============================] - ETA: 0s - loss: 0.3596 - accuracy: 0.9299
Epoch 1: saving model to BiLSTM_model_artistic.h5
313/313 [==============================] - 203s 628ms/step - loss: 0.3596 - accuracy: 0.9299 - val_loss: 0.1949 - val_accuracy: 0.9428
Epoch 2/10
313/313 [==============================] - ETA: 0s - loss: 0.1411 - accuracy: 0.9549
Epoch 2: saving model to BiLSTM_model_artistic.h5
313/313 [==============================] - 192s 613ms/step - loss: 0.1411 - accuracy: 0.9549 - val_loss: 0.1687 - val_accuracy: 0.9480
Epoch 3/10
313/313 [==============================] - ETA: 0s - loss: 0.1259 - accuracy: 0.9587
Epoch 3: saving model to BiLSTM_model_artistic.h5
313/313 [==============================] - 204s 652ms/step - loss: 0.1259 - accuracy: 0.9587 - val_loss: 0.1629 - val_accuracy: 0.9497
Epoch 4/10
313/313 [==============================] - ETA: 0s - loss: 0.1215 - accuracy: 0.9594
Epoch 4: saving model to BiLSTM_model_artis

Prediction Phase

In [ ]:
r.history['loss']

[0.35957053303718567,
 0.14108815789222717,
 0.12585732340812683,
 0.12151019275188446,
 0.11843199282884598,
 0.1161467656493187,
 0.11397743970155716,
 0.11195237934589386,
 0.11020161211490631,
 0.10853244364261627]

In [ ]:
r.history['val_loss']

[0.19487278163433075,
 0.16865253448486328,
 0.16291475296020508,
 0.16097094118595123,
 0.15967343747615814,
 0.1584356278181076,
 0.1581593006849289,
 0.15716703236103058,
 0.15765069425106049,
 0.1572256088256836]

In [ ]:
r.history['val_accuracy']

[0.9427601099014282,
 0.9480058550834656,
 0.9496764540672302,
 0.950194776058197,
 0.9503509998321533,
 0.9508346319198608,
 0.9507274627685547,
 0.9508797526359558,
 0.9510581493377686,
 0.9515072107315063]

In [ ]:
with open(f'results.pkl', 'wb') as file:
  pickle.dump([r.history['loss'], r.history['val_loss'], r.history['val_accuracy']], file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import numpy as np

In [ ]:
def convert_text_to_sequences(text):
  text_sequences = tokenizer_input.texts_to_sequences(text)
  print(text_sequences)
  text_sequences = pad_sequences(text_sequences, maxlen=max_len_input, padding='post', value=0)
  return text_sequences

In [ ]:
def predict(input_texts:list):
    sequence = convert_text_to_sequences(input_texts)
    p = np.argmax(model.predict(sequence), axis=-1)
    output_texts = []
    for i in range(len(input_texts)):
        predict_output = ""
        user_query_split = remove_punctuation(input_texts[i])
        user_query_split = user_query_split.lower().split(" ")
        for t in range(len(user_query_split)):
            try:
                predict_output += str(idx2word[user_query_split[t]][p[i][t]]) + " "
            except KeyError:
                predict_output += user_query_split[t] + " "
        output_texts.append(predict_output[:-1])
    return output_texts